## Notes
### Check sl bg fakeids

In [22]:
import numpy as np
import os
import pandas as pd
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [16]:
dirWork         = os.getcwd()
year            = '2019'
#dirInput        = os.path.join(dirWork, r'data\data' + year                            )

dirLEHD         = os.path.join(dirWork, r'results\results' + year + '\LODES'            )
dirSL           = os.path.join(dirWork, r'results\results' + year + '\SL'               ) 
dirIntermediate = os.path.join(dirWork, 'intermediate\intermediate' + year + '\LEHDVSL')
dirResults      = os.path.join(dirWork, r'results\results' + year + '\LEHDVSL'         )

print("Working Directory: "      + dirWork        )
#print("Input Directory: "        + dirInput       )
print("Intermediate Directory: " + dirIntermediate)
print("Results Directory: "      + dirResults     )

Working Directory: c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns
Intermediate Directory: c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\intermediate\intermediate2019\LEHDVSL
Results Directory: c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\results\results2019\LEHDVSL


In [111]:
#input filenames
gdb_LEHDCommutePatterns = "LEHDCommutePatterns.gdb"
gdb_SLCommutePatterns = "SLCommutePatterns.gdb"
fname_LEHDCommutePatterns_gdb = os.path.join(dirLEHD, gdb_LEHDCommutePatterns)
fname_SLCommutePatterns_gdb = os.path.join(dirSL, gdb_SLCommutePatterns)

lehd_BlockGroupsWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'LEHD_BlockGroup_CommutePatterns_Number')
lehd_TractsWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'LEHD_Tract_CommutePatterns_Number')
lehd_SDWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'LEHD_SD_CommutePatterns_Number')
lehd_CitiesWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'LEHD_City_CommutePatterns_Number')

sl_BlockGroupsWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'SL_BlockGroup_CommutePatterns_Number')
sl_TractsWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'SL_Tract_CommutePatterns_Number')
sl_SDWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'SL_SD_CommutePatterns_Number')
sl_CitiesWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'SL_City_CommutePatterns_Number')

In [107]:
#output filenames
gdb_LEHDVSLCommutePatterns = "LEHDVSLCommutePatterns.gdb"
fname_LEHDVSLCommutePatterns_gdb = os.path.join(dirResults, gdb_LEHDVSLCommutePatterns)

fname_BlockGroupsWCommutePatterns_Number = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_BlockGroup_CommutePatterns_Number')
fname_TractsWCommutePatterns_Number = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_Tract_CommutePatterns_Number')
fname_SDWCommutePatterns_Number = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_SD_CommutePatterns_Number')
fname_CitiesWCommutePatterns_Number = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_City_CommutePatterns_Number')

fname_BlockGroupsWCommutePatterns_PercDiff = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_BlockGroup_CommutePatterns_PercDiff')
fname_TractsWCommutePatterns_PercDiff = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_Tract_CommutePatterns_PercDiff')
fname_SDWCommutePatterns_PercDiff = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_SD_CommutePatterns_PercDiff')
fname_CitiesWCommutePatterns_PercDiff = os.path.join(fname_LEHDVSLCommutePatterns_gdb, 'LEHDVSL_City_CommutePatterns_PercDiff')

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

#if results geodatabase doesn't exist, create it
print("Checking if " + fname_LEHDVSLCommutePatterns_gdb + " exists...")
if not arcpy.Exists(os.path.join(dirResults, gdb_LEHDVSLCommutePatterns)):
    print("Creating " + fname_LEHDVSLCommutePatterns_gdb + "...")
    arcpy.management.CreateFileGDB(dirResults, gdb_LEHDVSLCommutePatterns)
else:
    print(gdb_LEHDVSLCommutePatterns + " exists...")

Checking if c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\results\results2019\LEHDVSL\LEHDVSLCommutePatterns.gdb exists...
LEHDVSLCommutePatterns.gdb exists...


In [126]:
def get_dif_between_layers(lehdLayer, slLayer, colname, range1, range2, range3):
    sdf_lehd = pd.DataFrame.spatial.from_featureclass(lehdLayer)
    sdf_sl = pd.DataFrame.spatial.from_featureclass(slLayer)

    # replace _o and _d with _h and _w in streetlight data
    sdf_sl.columns = sdf_sl.columns.str.replace('_o','_h',regex=True)
    sdf_sl.columns = sdf_sl.columns.str.replace('_d','_w',regex=True)

    # select needed columns only
    sdf_lehd_small = sdf_lehd.iloc[:, range1]
    sdf_sl_small = sdf_sl.iloc[:, range2]

    # set index columns
    sdf_lehd_small.set_index([colname,'SHAPE'], inplace=True)
    sdf_sl_small.set_index([colname,'SHAPE'], inplace=True)

    # get base columns to merge on later
    sdf_lehd_cols = sdf_lehd.iloc[:, range3]
    sdf_sl_cols = sdf_sl.iloc[:, range3]

    # calculate difference between dataframes
    sdf_diff = sdf_sl_small.subtract(sdf_lehd_small, fill_value=0).reset_index()

    # merge missing columns back on
    sdf_diff_full = sdf_diff.merge(sdf_lehd_cols, on = colname, how = 'left')

    return sdf_diff_full


def get_pc_between_layers(lehdLayer, slLayer, colname, range1, range2, range3):
    sdf_lehd = pd.DataFrame.spatial.from_featureclass(lehdLayer)
    sdf_sl = pd.DataFrame.spatial.from_featureclass(slLayer)

    # replace _o and _d with _h and _w in streetlight data
    sdf_sl.columns = sdf_sl.columns.str.replace('_o','_h',regex=True)
    sdf_sl.columns = sdf_sl.columns.str.replace('_d','_w',regex=True)

    # select needed columns only
    sdf_lehd_small = sdf_lehd.iloc[:, range1]
    sdf_sl_small = sdf_sl.iloc[:, range2]

    # set index columns
    sdf_lehd_small.set_index([colname,'SHAPE'], inplace=True)
    sdf_sl_small.set_index([colname,'SHAPE'], inplace=True)

    # get base columns to merge on later
    sdf_lehd_cols = sdf_lehd.iloc[:, range3]
    sdf_sl_cols = sdf_sl.iloc[:, range3]

    # calculate difference and pc between dataframes
    sdf_diff = sdf_sl_small.subtract(sdf_lehd_small, fill_value=0)
    sdf_pc = sdf_diff.div(sdf_lehd_small,fill_value = 0).mul(100)

    # reset index
    sdf_pc = sdf_pc.reset_index()

    # merge missing columns back on
    sdf_pc_full = sdf_pc.merge(sdf_lehd_cols, on = colname, how = 'left')

    return sdf_pc_full

    


In [123]:
sdf_bg_diff = get_dif_between_layers(lehd_BlockGroupsWCommutePatterns_Number, sl_BlockGroupsWCommutePatterns_Number, 'GEOID', np.r_[7,18:236], np.r_[7,18:236], np.r_[0:17])
sdf_bg_pc   = get_pc_between_layers(lehd_BlockGroupsWCommutePatterns_Number, sl_BlockGroupsWCommutePatterns_Number, 'GEOID', np.r_[7,18:236], np.r_[7,18:236], np.r_[0:17])

In [141]:
sdf_bg_diff.spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_Number, sanitize_columns=False)
sdf_bg_pc.spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_PercDiff, sanitize_columns=False)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2019\\LEHDVSL\\LEHDVSLCommutePatterns.gdb\\LEHDVSL_BlockGroup_CommutePatterns_PercDiff'

In [127]:
sdf_tract_diff= get_dif_between_layers(lehd_TractsWCommutePatterns_Number, sl_TractsWCommutePatterns_Number, 'GEOID20', np.r_[5,18:235],  np.r_[5,18:234], np.r_[0:17])
sdf_tract_pc  = get_pc_between_layers(lehd_TractsWCommutePatterns_Number, sl_TractsWCommutePatterns_Number, 'GEOID20', np.r_[5,18:235],  np.r_[5,18:234], np.r_[0:17])

In [142]:
sdf_tract_diff.spatial.to_featureclass(fname_TractsWCommutePatterns_Number, sanitize_columns=False)
sdf_tract_pc.spatial.to_featureclass(fname_TractsWCommutePatterns_PercDiff, sanitize_columns=False)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2019\\LEHDVSL\\LEHDVSLCommutePatterns.gdb\\LEHDVSL_Tract_CommutePatterns_PercDiff'

In [139]:
sdf_sd_diff = get_dif_between_layers(lehd_SDWCommutePatterns_Number, sl_SDWCommutePatterns_Number, 'DSML_NAME', np.r_[3,8:225], np.r_[3,8:224], np.r_[0:7])
sdf_sd_pc   = get_pc_between_layers(lehd_SDWCommutePatterns_Number, sl_SDWCommutePatterns_Number, 'DSML_NAME', np.r_[3,8:225], np.r_[3,8:224], np.r_[0:7])

In [143]:
sdf_sd_diff.spatial.to_featureclass(fname_SDWCommutePatterns_Number, sanitize_columns=False)
sdf_sd_pc.spatial.to_featureclass(fname_SDWCommutePatterns_PercDiff, sanitize_columns=False)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2019\\LEHDVSL\\LEHDVSLCommutePatterns.gdb\\LEHDVSL_SD_CommutePatterns_PercDiff'

In [140]:
sdf_city_diff = get_dif_between_layers(lehd_CitiesWCommutePatterns_Number, sl_CitiesWCommutePatterns_Number, 'CODE3', np.r_[1,4:222], np.r_[1,4:221], np.r_[0:3])
sdf_city_pc   = get_pc_between_layers(lehd_CitiesWCommutePatterns_Number, sl_CitiesWCommutePatterns_Number, 'CODE3', np.r_[1,4:222], np.r_[1,4:221], np.r_[0:3])

In [144]:
sdf_city_diff.spatial.to_featureclass(fname_CitiesWCommutePatterns_Number, sanitize_columns=False)
sdf_city_pc.spatial.to_featureclass(fname_CitiesWCommutePatterns_PercDiff, sanitize_columns=False)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2019\\LEHDVSL\\LEHDVSLCommutePatterns.gdb\\LEHDVSL_City_CommutePatterns_PercDiff'

### Testing


In [138]:
sdf_lehd_city_number = pd.DataFrame.spatial.from_featureclass(lehd_CitiesWCommutePatterns_Number)
sdf_sl_city_number = pd.DataFrame.spatial.from_featureclass(sl_CitiesWCommutePatterns_Number)

# replace _o and _d with _h and _w in streetlight data
sdf_sl_city_number.columns = sdf_sl_city_number.columns.str.replace('_o','_h',regex=True)
sdf_sl_city_number.columns = sdf_sl_city_number.columns.str.replace('_d','_w',regex=True)

# select needed columns only
sdf_lehd_city_number_small = sdf_lehd_city_number.iloc[:, np.r_[1,4:222]]
sdf_sl_city_number_small = sdf_sl_city_number.iloc[:, np.r_[1,4:221]]

# set index columns
sdf_lehd_city_number_small.set_index(['CODE3','SHAPE'], inplace=True)
sdf_sl_city_number_small.set_index(['CODE3','SHAPE'], inplace=True)

In [96]:
sdf_lehd_bg_number = pd.DataFrame.spatial.from_featureclass(lehd_BlockGroupsWCommutePatterns_Number)
sdf_sl_bg_number = pd.DataFrame.spatial.from_featureclass(sl_BlockGroupsWCommutePatterns_Number)

# replace _o and _d with _h and _w in streetlight data
sdf_sl_bg_number.columns = sdf_sl_bg_number.columns.str.replace('_o','_h',regex=True)
sdf_sl_bg_number.columns = sdf_sl_bg_number.columns.str.replace('_d','_w',regex=True)

# select needed columns only
sdf_lehd_bg_number_small = sdf_lehd_bg_number.iloc[:, np.r_[7,18:236]]
sdf_sl_bg_number_small = sdf_sl_bg_number.iloc[:, np.r_[7,18:236]]

# set index columns
sdf_lehd_bg_number_small.set_index(['GEOID','SHAPE'], inplace=True)
sdf_sl_bg_number_small.set_index(['GEOID','SHAPE'], inplace=True)

# get base columns to merge on later
sdf_lehd_bg_base_cols = sdf_lehd_bg_number.iloc[:, np.r_[0:17]]
sdf_sl_bg_base_cols = sdf_sl_bg_number.iloc[:, np.r_[0:17]]

# display tables
#display(sdf_lehd_bg_number_small)
#display(sdf_sl_bg_number_small)

In [97]:
sdf_bg_number_diff = sdf_sl_bg_number_small.subtract(sdf_lehd_bg_number_small, fill_value=0)
sdf_bg_number_pc = sdf_bg_number_diff.div(sdf_lehd_bg_number_small,fill_value = 0).mul(100)

sdf_bg_number_diff = sdf_bg_number_diff.reset_index()
sdf_bg_number_pc = sdf_bg_number_pc.reset_index()

In [99]:
sdf_bg_number_diff_full = sdf_bg_number_diff.merge(sdf_lehd_bg_base_cols, on = 'GEOID', how = 'left')
sdf_bg_number_pc_full = sdf_bg_number_pc.merge(sdf_lehd_bg_base_cols, on = 'GEOID', how = 'left')

In [200]:
#input filenames
fname_CityTownship_csv        = os.path.join(dirInput, 'citytownship.csv'                                         )
fname_CityTownship_shp        = os.path.join(dirInput, 'Municipalities_Township' + r'\Municipalities_Township.shp') 
fname_SLCommutePatterns_csv = os.path.join(dirInput, 'sl_at_blockgroup_level' + year + '.csv'                   )
#fname_UofUManualAllocation_csv= os.path.join(dirInput, '490351108004_Manual_Allocation.csv'                      )
fname_BlockGroups_shp         = os.path.join(dirInput, 'tl_2020_49_bg' + r'\tl_2020_49_sd_bg.shp'                 )
fname_Counties_AGRC_shp       = os.path.join(dirInput, 'Utah_County_Boundaries-shp' + r'\Counties.shp'            )
fname_SLCommutePatterns_csv

#intermediate
fname_CityTownshipCounty_BlockGroups_shp = os.path.join(dirIntermediate, 'citytownshipcounty_blockgroup_intersect.shp')
fname_CityTownshipWCodes_shp             = os.path.join(dirIntermediate, 'Municipalities_Townships_withCodes.shp')
fname_CountyWCodes_shp                   = os.path.join(dirIntermediate, 'Counties_withCodes.shp')
fname_CityTownshipWCodesUnionCounty_shp  = os.path.join(dirIntermediate, 'Municipalities_Townships_withCodes_Counties.shp')
fname_CityTownshipCountyWCodes_shp       = os.path.join(dirIntermediate, 'Municipalities_Townships_Counties_withCodes.shp')

#display(dirIntermediate)

#results geodatabase

fname_CityTownshipJSON = os.path.join(dirResults, "sl_citytownshipdata.json")

gdb_SLCommutePatterns = "SLCommutePatterns.gdb"
fname_SLCommutePatterns_gdb = os.path.join(dirResults, gdb_SLCommutePatterns)

fname_CityTownshipWCodes = os.path.join(fname_SLCommutePatterns_gdb, 'Municipalities_and_Townships')
#fname_CityTownshipCounty = os.path.join(fname_SLCommutePatterns_gdb, 'Municipalities_Townships_Counties')

fname_BlockGroupsWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_BlockGroup_CommutePatterns_Number')
fname_BlockGroupsWCommutePatterns_Percent = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_BlockGroup_CommutePatterns_Percent')

fname_TractsWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_Tract_CommutePatterns_Number')
fname_TractsWCommutePatterns_Percent = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_Tract_CommutePatterns_Percent')

fname_SDWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_SD_CommutePatterns_Number')
fname_SDWCommutePatterns_Percent = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_SD_CommutePatterns_Percent')

fname_CitiesWCommutePatterns_Number = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_City_CommutePatterns_Number')
fname_CitiesWCommutePatterns_Percent = os.path.join(fname_SLCommutePatterns_gdb, 'LEHDVSL_City_CommutePatterns_Percent')

fname_Counties = os.path.join(fname_SLCommutePatterns_gdb, 'Counties')

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

#if results geodatabase doesn't exist, create it
print("Checking if " + fname_SLCommutePatterns_gdb + " exists...")
if not arcpy.Exists(os.path.join(dirResults, gdb_SLCommutePatterns)):
    print("Creating " + fname_SLCommutePatterns_gdb + "...")
    arcpy.management.CreateFileGDB(dirResults, gdb_SLCommutePatterns)
else:
    print(gdb_SLCommutePatterns + " exists...")

#print(AnalysisAreasInput)

Checking if c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\results\results2019\SL\SLCommutePatterns.gdb exists...
Creating c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\results\results2019\SL\SLCommutePatterns.gdb...


# Create Cities/Townships Feature Class with 3-Digit Code and County FIPS